In [ ]:
import import_ipynb
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

In [ ]:
import tensorflow as tf

print (tf.__version__)

In [ ]:
#To know if your system is built with CUDA and using gpu or not.

print ("Usage of gpu: {}".format(tf.test.is_gpu_available())) 
print ("Built with CUDA: {}".format(tf.test.is_built_with_cuda()))

## Loading the train set images and respective masked images

In [ ]:
x_train_img,x_val_img=[],[]
y_train_mask,y_val_mask=[],[]

In [ ]:
import data
x_train_img,y_train_mask=data.train_value()

## Standardizing the train images and the masked images

In [ ]:
x_train_img=np.asarray(x_train_img)
y_train_mask=np.asarray(y_train_mask)

x_train_img_m=np.mean(x_train_img,axis=0,keepdims=True)
x_train_img_s=np.std(x_train_img,axis=0,keepdims=True)

x_train_img=(x_train_img-x_train_img_m)/x_train_img_s
y_train_mask=y_train_mask*(1./255)

In [ ]:
print(x_train_img.shape,y_train_mask.shape) 

In [ ]:
import data1
x_val_img,y_val_mask=data1.val_value()

## Standardizing the validation set images and the validation  masked images

In [ ]:
x_val_img=np.asarray(x_val_img)
y_val_mask=np.asarray(y_val_mask)

x_val_img_m=np.mean(x_val_img,axis=0,keepdims=True)
x_val_img_s=np.std(x_val_img,axis=0,keepdims=True)

x_val_img=(x_val_img-x_val_img_m)/x_val_img_s
y_val_mask=y_val_mask*(1./255)

In [ ]:
print (x_val_img.shape,y_val_mask.shape)

#### importing the necessary functions from tensorflow's keras API to construct my U-NET architecture

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D,BatchNormalization,Activation,MaxPooling2D,Input,Flatten,Dropout,Conv2DTranspose,concatenate
from keras import optimizers
from keras import backend as K
import cv2 as cv

#### Designing custom metric (dice coefficient) as a performance metric and custom loss function(dice loss) to upgrade the parametrics   

In [ ]:
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + 1) / (K.sum(y_true_f**2) + K.sum(y_pred_f**2) + 1)

In [ ]:
def dice_loss(y_true,y_pred):
    return 1-dice_coef(y_true,y_pred)

In [ ]:
def conv2d_block(input_tensor, n_filters, kernel_size = 3, batchnorm = True):
    """Function to add 2 convolutional layers with the parameters passed to it"""
    # first layer
    x = Conv2D(filters = n_filters, kernel_size = (kernel_size, kernel_size),\
              kernel_initializer = 'he_normal', padding = 'same')(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    # second layer
    x = Conv2D(filters = n_filters, kernel_size = (kernel_size, kernel_size),\
              kernel_initializer = 'he_normal', padding = 'same')(x)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    return x

## Creating the UNET architecture

In [ ]:
def get_unet(input_img, n_filters = 32, dropout = 0.2, batchnorm = True):
    # Contracting Path
    c1 = conv2d_block(input_img, n_filters * 1, kernel_size = 3, batchnorm = batchnorm)
    p1 = MaxPooling2D((2, 2))(c1)
    p1 = Dropout(dropout)(p1)
    
    c2 = conv2d_block(p1, n_filters * 2, kernel_size = 3, batchnorm = batchnorm)
    p2 = MaxPooling2D((2, 2))(c2)
    p2 = Dropout(dropout)(p2)
    
    c3 = conv2d_block(p2, n_filters * 4, kernel_size = 3, batchnorm = batchnorm)
    p3 = MaxPooling2D((2, 2))(c3)
    p3 = Dropout(dropout)(p3)
    
    c4 = conv2d_block(p3, n_filters * 8, kernel_size = 3, batchnorm = batchnorm)
    p4 = MaxPooling2D((2, 2))(c4)
    p4 = Dropout(dropout)(p4)
    
    c5 = conv2d_block(p4, n_filters = n_filters * 16, kernel_size = 3, batchnorm = batchnorm)
    
    # Expansive Path
    u6 = Conv2DTranspose(n_filters * 8, (3, 3), strides = (2, 2), padding = 'same')(c5)
    u6 = concatenate([u6, c4])
    u6 = Dropout(dropout)(u6)
    c6 = conv2d_block(u6, n_filters * 8, kernel_size = 3, batchnorm = batchnorm)
    
    u7 = Conv2DTranspose(n_filters * 4, (3, 3), strides = (2, 2), padding = 'same')(c6)
    u7 = concatenate([u7, c3])
    u7 = Dropout(dropout)(u7)
    c7 = conv2d_block(u7, n_filters * 4, kernel_size = 3, batchnorm = batchnorm)
    
    u8 = Conv2DTranspose(n_filters * 2, (3, 3), strides = (2, 2), padding = 'same')(c7)
    u8 = concatenate([u8, c2])
    u8 = Dropout(dropout)(u8)
    c8 = conv2d_block(u8, n_filters * 2, kernel_size = 3, batchnorm = batchnorm)
    
    u9 = Conv2DTranspose(n_filters * 1, (3, 3), strides = (2, 2), padding = 'same')(c8)
    u9 = concatenate([u9, c1])
    u9 = Dropout(dropout)(u9)
    c9 = conv2d_block(u9, n_filters * 1, kernel_size = 3, batchnorm = batchnorm)
    
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c9)
    model = Model(inputs=input_img, outputs=outputs)
    return model

# Preparation for training

In [ ]:
model=get_unet(input_img=Input(shape=(256,256,1)))

In [ ]:
model.summary()

In [ ]:
adam = optimizers.Adam(learning_rate=0.01)
model.compile(optimizer='adam',loss=dice_loss,metrics=[dice_coef])

In [ ]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
model_path="{}_best.hdf5".format('my_model')

checkpoint = ModelCheckpoint(model_path, monitor='val_loss', verbose=1,save_best_only=True, mode='min', save_weights_only = False)

reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.5,patience=3,verbose=1, mode='min', epsilon=0.0001, cooldown=1, min_lr=1e-7)
early = EarlyStopping(monitor="val_loss",mode="min",patience=7) 
callbacks_list = [checkpoint, early, reduceLROnPlat]

## Starting the training process

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
data_gen_args = dict(rotation_range=90,width_shift_range=0.1,height_shift_range=0.1,zoom_range=0.2)

image_datagen = ImageDataGenerator(**data_gen_args)


model.fit(image_datagen.flow(x_train_img,y_train_mask,batch_size=8),epochs=57,callbacks=callbacks_list,validation_data=(x_val_img,y_val_mask))

In [ ]:
##loading saved model to see its performance
import cv2 as cv
from tensorflow.keras.models import load_model
modelL=load_model('my_model_best.hdf5',custom_objects={'dice_loss':dice_loss,'dice_coef':dice_coef})

In [ ]:
test_img=x_val_img[np.random.randint(low=0,high=141)] #choosing a random picture from valid set to predict its mask.
test_img_final=np.expand_dims(test_img,axis=0)        #expanding its dimension so its as per the model's architecture.
test_img=np.squeeze(test_img)

In [ ]:
pred_img=modelL.predict(test_img_final)              #predicting the masked image for the given test_img

In [ ]:
pred_img=np.squeeze(pred_img)
pred_img=pred_img>0.5                                #using 0.5 as threshold where values >0.5 are 1 and rest as 0's.
pred_img=pred_img*255.0

In [ ]:
#displaying the masked output over the gryscale image as a transparent layer

im1 = plt.imshow(test_img, cmap="gray")
im2 = plt.imshow(pred_img,cmap='Greens',alpha=0.2)
plt.show()